*Lab | Making predictions with logistic regression*


1. Create a query or queries to extract the information you think may be relevant for building the prediction model. It should include some film features and some rental features (X).

In [242]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import getpass  # To get the password without showing the input
# password = getpass.getpass()

In [243]:
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)
query = '''SELECT f.title, r.rental_date, f.rating, f.length, f.rental_duration, f.special_features, fc.category_id
FROM film f
LEFT JOIN inventory i
USING (film_id)
LEFT JOIN rental r
USING (inventory_id)
LEFT JOIN film_category fc
USING (film_id)
GROUP BY f.title
;'''

data = pd.read_sql_query(query, engine)
data.head()

,title,rental_date,rating,length,rental_duration,special_features,category_id
0,ACADEMY DINOSAUR,2005-07-08 19:03:15,PG,86,6,"Deleted Scenes,Behind the Scenes",6
1,ACE GOLDFINGER,2005-08-01 04:24:47,G,48,3,"Trailers,Deleted Scenes",11
2,ADAPTATION HOLES,2005-07-27 01:21:19,NC-17,50,7,"Trailers,Deleted Scenes",6
3,AFFAIR PREJUDICE,2005-07-12 02:40:41,G,117,5,"Commentaries,Behind the Scenes",11
4,AFRICAN EGG,2005-05-28 07:53:38,G,130,6,Deleted Scenes,8


In [244]:
data.isna().sum()

title                0
rental_date         42
rating               0
length               0
rental_duration      0
special_features     0
category_id          0
dtype: int64

2. Create a query to get the list of all unique film titles and a boolean indicating if it was rented (rental_date) in May 2005. (Create new column called - 'rented_in_may'). This will be our TARGET (y) variable.

3. Read the data into a Pandas dataframe. At this point you should have 1000 rows. Number of columns depends on the number of features you chose.

In [245]:
import numpy as np
data['rented_in_may']=np.where((data['rental_date'].dt.month)==5.0, True, False)
data.head()

,title,rental_date,rating,length,rental_duration,special_features,category_id,rented_in_may
0,ACADEMY DINOSAUR,2005-07-08 19:03:15,PG,86,6,"Deleted Scenes,Behind the Scenes",6,False
1,ACE GOLDFINGER,2005-08-01 04:24:47,G,48,3,"Trailers,Deleted Scenes",11,False
2,ADAPTATION HOLES,2005-07-27 01:21:19,NC-17,50,7,"Trailers,Deleted Scenes",6,False
3,AFFAIR PREJUDICE,2005-07-12 02:40:41,G,117,5,"Commentaries,Behind the Scenes",11,False
4,AFRICAN EGG,2005-05-28 07:53:38,G,130,6,Deleted Scenes,8,True


In [246]:
data.dtypes

title                       object
rental_date         datetime64[ns]
rating                      object
length                       int64
rental_duration              int64
special_features            object
category_id                  int64
rented_in_may                 bool
dtype: object

In [247]:
data['rented_in_may'].value_counts()

False    775
True     225
Name: rented_in_may, dtype: int64

In [248]:
# drop rental_date
data = data.drop('rental_date', axis=1)
data

,title,rating,length,rental_duration,special_features,category_id,rented_in_may
0,ACADEMY DINOSAUR,PG,86,6,"Deleted Scenes,Behind the Scenes",6,False
1,ACE GOLDFINGER,G,48,3,"Trailers,Deleted Scenes",11,False
2,ADAPTATION HOLES,NC-17,50,7,"Trailers,Deleted Scenes",6,False
3,AFFAIR PREJUDICE,G,117,5,"Commentaries,Behind the Scenes",11,False
4,AFRICAN EGG,G,130,6,Deleted Scenes,8,True
...,...,...,...,...,...,...,...
995,YOUNG LANGUAGE,G,183,6,"Trailers,Behind the Scenes",6,False
996,YOUTH KICK,NC-17,179,4,"Trailers,Behind the Scenes",12,False
997,ZHIVAGO CORE,NC-17,105,6,Deleted Scenes,11,False
998,ZOOLANDER FICTION,R,101,5,"Trailers,Deleted Scenes",3,False


4. Analyze extracted features (X) and transform them. You may need to encode some categorical variables, or scale numerical variables.

In [249]:
# # split special features into separate columns
# split = pd.DataFrame(data['special_features'].values.tolist()).rename(columns={0:'special_features'})
# split = split['special_features'].str.split(',', expand=True)

In [250]:
# split[0].value_counts()

In [251]:
# split.columns = ['sf1', 'sf2', 'sf3', 'sf4']
# display(split.head(1))

In [252]:
# data = pd.concat([data, split], axis=1)

In [253]:
# data = data.drop('special_features',axis=1)
# data

In [255]:
# convert category_id into string
data['category_id'] = data['category_id'].astype(str)
data['category_id'].dtype

dtype('O')

In [256]:
# count numbers of special features
sf_count = []
for index, row in data.iterrows():

    count = len(row["special_features"].split(","))
    sf_count.append(count)


data["special_feature_count"] = sf_count
data = data.drop('special_features', axis=1)
data

,title,rating,length,rental_duration,category_id,rented_in_may,special_feature_count
0,ACADEMY DINOSAUR,PG,86,6,6,False,2
1,ACE GOLDFINGER,G,48,3,11,False,2
2,ADAPTATION HOLES,NC-17,50,7,6,False,2
3,AFFAIR PREJUDICE,G,117,5,11,False,2
4,AFRICAN EGG,G,130,6,8,True,1
...,...,...,...,...,...,...,...
995,YOUNG LANGUAGE,G,183,6,6,False,2
996,YOUTH KICK,NC-17,179,4,12,False,2
997,ZHIVAGO CORE,NC-17,105,6,11,False,1
998,ZOOLANDER FICTION,R,101,5,3,False,2


In [266]:
# X-y split AND train-test split
y = data['rented_in_may']
X = data.drop('rented_in_may', axis=1)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1337)

In [267]:
from sklearn.preprocessing import MinMaxScaler
# from sklearn.preprocessing import StandardScaler

X_train_num = X_train.select_dtypes(include = np.number)

# Scaling data
transformer = MinMaxScaler().fit(X_train_num) # need to keep transformer
X_train_normalized = transformer.transform(X_train_num)
X_train_norm = pd.DataFrame(X_train_normalized)

In [268]:
X_train_norm.columns = X_train_num.columns
X_train_norm.head()

,length,rental_duration,special_feature_count
0,0.769784,0.50,0.000000
1,0.151079,0.75,0.000000
2,0.258993,0.00,0.333333
3,0.223022,0.50,0.000000
4,0.733813,0.25,0.333333


In [300]:
X_train_categorical = X_train.select_dtypes(include = object).drop('title', axis=1)
X_train_cat = pd.get_dummies(X_train_categorical, 
                             columns=['rating', 'category_id'],
                             drop_first=True)
X_train_cat.head()

,rating_NC-17,rating_PG,rating_PG-13,rating_R,category_id_10,category_id_11,category_id_12,category_id_13,category_id_14,category_id_15,category_id_16,category_id_2,category_id_3,category_id_4,category_id_5,category_id_6,category_id_7,category_id_8,category_id_9
46,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
789,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
722,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
283,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
39,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [264]:
# onehotencoding rating & category_id
# from sklearn.preprocessing import OneHotEncoder
# encoder = OneHotEncoder(drop='first').fit(X_train_cat)
# cols = encoder.get_feature_names_out(input_features=X_train_cat.columns)
# X_train_cat_encode = pd.DataFrame(encoder.transform(X_train_cat).toarray(),columns=cols)
# X_train_cat_encode.shape


(800, 19)

In [273]:
X_train_transformed = np.concatenate([X_train_norm, X_train_cat], axis=1)

5. Create a logistic regression model to predict 'rented_in_may' from the cleaned data.
Evaluate the results.

In [287]:
from sklearn.linear_model import LogisticRegression
classification = LogisticRegression(random_state=1337, solver='saga',
                  multi_class='multinomial').fit(X_train_transformed, y_train)

In [288]:
classification = LogisticRegression(random_state=1337, solver='saga',
                  multi_class='multinomial').fit(X_train_transformed, y_train)

In [289]:
X_test_num = X_test.select_dtypes(include = np.number)
X_test_normalized = transformer.transform(X_test_num)
X_test_norm = pd.DataFrame(X_test_normalized)


In [290]:
X_test_categorical = X_test.select_dtypes(include = object).drop('title', axis=1)
X_test_cat = pd.get_dummies(X_test_categorical, 
                             columns=['rating', 'category_id'],
                             drop_first=True)
X_test_cat.shape

(200, 19)

In [291]:
X_test_transformed = np.concatenate([X_test_norm, X_test_cat], axis=1)

In [301]:
X_test_transformed

array([[0.37410072, 0.        , 0.33333333, ..., 0.        , 0.        ,
        0.        ],
       [0.96402878, 0.75      , 0.33333333, ..., 0.        , 0.        ,
        1.        ],
       [0.31654676, 0.25      , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.27338129, 1.        , 0.33333333, ..., 0.        , 0.        ,
        0.        ],
       [0.29496403, 0.25      , 0.33333333, ..., 0.        , 0.        ,
        0.        ],
       [0.4028777 , 0.5       , 0.33333333, ..., 0.        , 0.        ,
        1.        ]])

In [292]:
predictions = classification.predict(X_test_transformed)
classification.score(X_test_transformed, y_test)

0.745

In [293]:
print(y_test.value_counts())

False    149
True      51
Name: rented_in_may, dtype: int64


In [294]:
pd.Series(predictions).value_counts()

False    200
dtype: int64

In [295]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, predictions)

array([[149,   0],
       [ 51,   0]])

In [296]:
from sklearn import neighbors
clf = neighbors.KNeighborsClassifier(n_neighbors=5, weights='uniform')
clf.fit(X_train_transformed, y_train)
predictions_clf = clf.predict(X_test_transformed)
clf.score(X_test_transformed, y_test)

0.705

In [297]:
confusion_matrix(y_test, predictions_clf)

array([[139,  10],
       [ 49,   2]])

In [298]:
pd.Series(y_test).value_counts()

False    149
True      51
Name: rented_in_may, dtype: int64

In [299]:
pd.Series(predictions_clf).value_counts()

False    188
True      12
dtype: int64